In [1]:
# Importēt nepieciešamās bibliotēkas

import gzip
# from rdflib.namespace import RDF
# from rdflib import Dataset, Namespace, RDF, RDFS, URIRef, Literal, BNode
from collections import Counter

In [2]:
# Funkcija, kas izveido RDF četrinieku sarakstu no .gz faila vai failiem
# Jāpadod faila vārds un Schema.org klases vārds, piemēram, School

def parse_gz_files(filenames, schema_org_class_name):
    current_subject_has_class = False
    quads = []
    
    for filename in filenames:
        with gzip.open(filename, 'rt', encoding='utf-8') as f:
            for line in f:

                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                
                try:
                    s, p, o, g, _ = line.split()
                except ValueError:
                    continue

                # Datu kopā RDF četriniekiem ir šāda struktūra:
                # Sākumā nāk četrinieks, kas norāda aprakstamās entītijas tipu
                # Tam seko pārējie četrinieki, kas apraksta entītijas īpašības
                # Tātad, datu apstrāde notiek blokos: sākumā nolasām tipu, tad nolasām īpašības

                # Datu kopa satur ne tikai entītijas ar tipu X, bet arī citas
                # Mēs gribam analizēt tikai entītijas ar tipu X
                # Lai noskaidrotu tipu, mēs nolasām predikātu, kas apzīmē tipu, un pārbaudām, vai tā vērtība atbilst schema.org tipam X
                # Ja tips nav X, tad datu bloks tiek ignorēts

                if p == "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
                    if o == f"<http://schema.org/{schema_org_class_name}>":
                        current_subject_has_class = True
                    else:
                        current_subject_has_class = False

                if current_subject_has_class:
                    quads.append((s, p, o, g))
    
    return quads


In [ ]:
# Iegūt četriniekus klasei Hospital 2021., 2020. un 2019. gadu datu kopās

hospital_quads = {}
hospital_quads["2019"] = parse_gz_files(["../hospital_2019.gz"], "Hospital")
hospital_quads["2020"] = parse_gz_files(["../hospital_2020.gz"], "Hospital")
hospital_quads["2021"] = parse_gz_files(["../hospital_2021.gz"], "Hospital")

In [5]:
for year, quads in hospital_quads.items():
    # Top 10 predikāti (predicates) entītijām ar tipu School
    
    predicate_counter = Counter()

    for s, p, o, graph in quads:
        predicate_counter[p] += 1

    top_predicates = predicate_counter.most_common(10)

    print(f"\nTop 10 predikāti {year}:")

    # Īpašība; entītiju skaits, kas izmanto; procents no kopējā entītiju skaita
    # Tā kā katra entītija sākas ar tipa predikātu, tipa predikātu skaitu var uzskatīt par entītiju skaitu
    for p, count in top_predicates:
        print(p, count, str(round(100 * (count / (predicate_counter["<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>"])), 2)) + "%")


Top 10 predikāti 2019:
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 141301 100.0%
<http://schema.org/Hospital/image> 101797 72.04%
<http://schema.org/Physician/hospitalAffiliation> 16554 11.72%
<http://schema.org/Hospital/url> 6336 4.48%
<http://www.w3.org/1999/xhtml/microdata#item> 5648 4.0%
<http://schema.org/Hospital/name> 2831 2.0%
<http://schema.org/Physician/mainEntityOfPage> 1982 1.4%
<https://schema.org/MedicalWebPage/about> 1623 1.15%
<http://schema.org/Hospital/address> 1083 0.77%
<http://schema.org/Hospital/legalName> 982 0.69%

Top 10 predikāti 2020:
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 271899 100.0%
<http://schema.org/address> 163767 60.23%
<http://schema.org/image> 139836 51.43%
<http://schema.org/name> 113111 41.6%
<http://www.w3.org/1999/xhtml/microdata#item> 105990 38.98%
<http://schema.org/telephone> 104280 38.35%
<http://schema.org/url> 79125 29.1%
<http://schema.org/priceRange> 55292 20.34%
<http://schema.org/hasMap> 47582 17.5%
<http://schema.org